In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import yfinance as yf

tickers = pd.read_csv('nasdaq_screener.csv')['Symbol'].values

predictions = {}
for i, ticker in enumerate(tickers):
    try:
        market = yf.Ticker(ticker).history(period='max')
        market.index = pd.to_datetime(market.index)
        market = market.drop(["Dividends", "Stock Splits"], axis=1)

        horizons = [2, 8, 32, 128, 512]
        predictors = []
        for horizon in horizons:
            rolling_averages = market.rolling(horizon).mean()
            ratio_column = f"Close_Ratio_{horizon}"
            market[ratio_column] = market["Close"] / rolling_averages["Close"]
            trend_column = f"Trend_{horizon}"
            market[trend_column] = market.shift(1).rolling(horizon).sum()["Close"].pct_change()
            predictors += [ratio_column, trend_column]

        market = market.dropna()

        market["Target"] = market["Close"].shift(-5) > market["Close"]

        model = RandomForestClassifier(n_estimators=250, min_samples_split=50, random_state=0)
        model.fit(market[predictors][:-5], market["Target"][:-5])

        last_data = market[predictors].iloc[-1:]
        prediction = model.predict_proba(last_data)[0, 1]

        if i % 100 == 0:
            print(f"{i} out of {len(tickers)} complete")
        predictions[ticker] = prediction

    except Exception as error:
        continue

sorted_preds = sorted(predictions.items(), key=lambda x:x[1])
sorted_preds[-10:]

0 out of 7087 complete


AACIW: Period 'max' is invalid, must be one of ['1d', '5d']
AAGRW: Period 'max' is invalid, must be one of ['1d', '5d']
ABLLW: Period 'max' is invalid, must be one of ['1d', '5d']
$ABR^D: possibly delisted; No timezone found
$ABR^E: possibly delisted; No timezone found
$ABR^F: possibly delisted; No timezone found
ABVEW: Period 'max' is invalid, must be one of ['1d', '5d']
ACONW: Period 'max' is invalid, must be one of ['1d', '5d']
$ACP^A: possibly delisted; No timezone found
$ACR^C: possibly delisted; No timezone found
$ACR^D: possibly delisted; No timezone found
$ADC^A: possibly delisted; No timezone found


100 out of 7087 complete


ADNWW: Period 'max' is invalid, must be one of ['1d', '5d']
ADSEW: Period 'max' is invalid, must be one of ['1d', '5d']
ADVWW: Period 'max' is invalid, must be one of ['1d', '5d']
AEAEW: Period 'max' is invalid, must be one of ['1d', '5d']
AENTW: Period 'max' is invalid, must be one of ['1d', '5d']
AERTW: Period 'max' is invalid, must be one of ['1d', '5d']
AFARW: Period 'max' is invalid, must be one of ['1d', '5d']
AFRIW: Period 'max' is invalid, must be one of ['1d', '5d']
AGBAW: Period 'max' is invalid, must be one of ['1d', '5d']
$AGM^C: possibly delisted; No timezone found
$AGM^D: possibly delisted; No timezone found
$AGM^E: possibly delisted; No timezone found
$AGM^F: possibly delisted; No timezone found
$AGM^G: possibly delisted; No timezone found
AGRIW: Period 'max' is invalid, must be one of ['1d', '5d']


200 out of 7087 complete


...

6900 out of 7087 complete


$WRB^E: possibly delisted; No timezone found
$WRB^F: possibly delisted; No timezone found
$WRB^G: possibly delisted; No timezone found
$WRB^H: possibly delisted; No timezone found
Failed to get ticker 'WSO/B' reason: Expecting value: line 1 column 1 (char 0)
$WSO/B: possibly delisted; No timezone found
XBPEW: Period 'max' is invalid, must be one of ['1d', '5d']
XFINW: Period 'max' is invalid, must be one of ['1d', '5d']
$XFLT^A: possibly delisted; No timezone found
XOSWW: Period 'max' is invalid, must be one of ['1d', '5d']


7000 out of 7087 complete


$YCBD^A: possibly delisted; No timezone found
ZAPPW: Period 'max' is invalid, must be one of ['1d', '5d']
ZCARW: Period 'max' is invalid, must be one of ['1d', '5d']
ZEOWW: Period 'max' is invalid, must be one of ['1d', '5d']
ZOOZW: Period 'max' is invalid, must be one of ['1d', '5d']
ZPTAW: Period 'max' is invalid, must be one of ['1d', '5d']
ZURAW: Period 'max' is invalid, must be one of ['1d', '5d']


[('TRIS', 0.8459400827994685),
 ('DECA', 0.8539999999999993),
 ('NVEI', 0.8573153888620794),
 ('SBBA', 0.8651400196656307),
 ('TME', 0.8704747394075828),
 ('NEWTL', 0.8804806528774317),
 ('RRAC', 0.8827393282106726),
 ('MCAA', 0.9091390681146115),
 ('APXI', 0.9238218547401222),
 ('STR', 0.9312000000000014)]